## Set up

In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Install the necessary libraries

In [ ]:
!pip install -qqq crewai_tools
!pip install -qqq crewai
!pip install -qqq openai

Load the api keys

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai-key')
os.environ["PINECONE_API_KEY"] = userdata.get('pinecone-key')
# Agent will automatically use the model defined in the environment variable
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4'

In [ ]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import PDFSearchTool
from dotenv import load_dotenv

load_dotenv()

False

## Creating Tools

In [ ]:
import os

print(os.listdir('/content'))


['.config', '.ipynb_checkpoints', 'drive', 'Intro_to_Utilitarianism.pdf', 'History_of_Olympics.pdf', 'db', 'Road_Trip.pdf', 'sample_data']


In [ ]:
pdf_search_tool1 = PDFSearchTool(
    pdf='/content/History_of_Olympics.pdf',
)

pdf_search_tool2 = PDFSearchTool(
    pdf='/content/Intro_to_Utilitarianism.pdf',
)

pdf_search_tool3 = PDFSearchTool(
    pdf='/content/Road_Trip.pdf',
)


In [ ]:
# from crewai_tools import RagTool

# # Example: Loading from a file
# rag_tool = RagTool()
# rag_tool.from_file('/content/Road_Trip.pdf')
# rag_tool = RagTool('/content/Road_Trip.pdf')
# rag_tool = RagTool().from_file('/content/Road_Trip.pdf')

In [ ]:
# user_query = "How far is Twin Falls, ID from Sacramento, CA?"
user_query = "When was the first Olympics held?"

## Creating Agents

In [ ]:
agent1 = Agent(
    role="pdf read Agent1",
    goal="Search through the PDF to find relevant answers to the query",
    verbose=True,
    memory = True,
    backstory=(
        """
        You are adept at searching and extracting data from first document
        in the directory, ensuring accurate and prompt responses.
        """
    ),
    # tools=[pdf_search_tool1],
    allow_delegation=False
)

agent2 = Agent(
    role="pdf read Agent2",
    goal="Search through the PDF to find relevant answers to the query",
    verbose=True,
    memory = True,
    backstory=(
        """
        You are adept at searching and extracting data from second document
        in the directory, ensuring accurate and prompt responses.
        """
    ),
    # tools=[pdf_search_tool2],
    allow_delegation=False
)

agent3 = Agent(
    role="pdf read Agent3",
    goal="Search through the PDF to find relevant answers to the query",
    verbose=True,
    memory = True,
    backstory=(
        """
        You are adept at searching and extracting data from third pdf document
        in the directory, ensuring accurate and prompt responses.
        """
    ),
    # tools=[pdf_search_tool3],
    allow_delegation=False
)

Create a manager agent for strategic task assignments

In [ ]:
# manager = Agent(
#     role ="Project Manager",
#     goal = "strategically & efficiently manage the tasks among the crew members & ensure high quality task completion",
#     backstory=(
#         """
#         You are an experienced project manager skiled in overseeing complex projects
#         and guiding teams to success.
#         Your goal is to coordinate efforts of the crew members, to extract ONLY
#         the relevant information from the 3 pdf documents provided to each other agents
#         in reponse to the user's query.

#         If you cannot find the information that the user is seeking for,
#         in any of the three pdfs, you respond saying that the query isn't relevant.

#         The User's query is:
#         {user_query}
#         """
#     ),
#     allow_delegation = True
# )

## Creating Tasks

In [ ]:
response_task1 = Task(
    description=(
        """
        Provide a response to user's query based on content of the provided PDF.
        The agent1 will search through the PDF to find the relevant answers.
        The final answer MUST be clear & accurate, based on the content of the PDF.

        The User's query is:
        {user_query}
        """
    ),
    expected_output="""
        Provide clear and accurate answers to the user's queries based on
        the content of the PDF.
        """,
    tools=[pdf_search_tool1],
    agent=agent1
)

response_task2 = Task(
    description=(
        """
        Provide a response to user's query based on content of the provided PDF.
        The agent1 will search through the PDF to find the relevant answers.
        The final answer MUST be clear & accurate, based on the content of the PDF.

        The User's query is:
        {user_query}
        """
    ),
    expected_output="""
        Provide clear and accurate answers to the user's queries based on
        the content of the PDF.
        """,
    tools=[pdf_search_tool2],
    agent=agent2
)

response_task3 = Task(
    description=(
        """
        Provide a response to user's query based on content of the provided PDF.
        The agent1 will search through the PDF to find the relevant answers.
        The final answer MUST be clear & accurate, based on the content of the PDF.

        The User's query is:
        {user_query}
        """
    ),
    expected_output="""
        Provide clear and accurate answers to the user's queries based on
        the content of the PDF.
        """,
    tools=[pdf_search_tool3],
    agent=agent3
)

## Creating Crew with a Custom Manager

When kicking off the crew, I got an error saying, "Manager agent should not have tools", although I haven't allocated any tools to the 'manager' agent.

Therefore, I am manually setting 'tools' attribute to an empty list here.

In [ ]:
manager.tools=[]

In [ ]:
from langchain_openai import ChatOpenAI

manager_llm = ChatOpenAI(model_name="gpt-4")

crew = Crew(
    agents=[agent1, agent2, agent3],
    tasks=[response_task1, response_task2, response_task3],
    # manager_agent=manager,
    manager_llm=manager_llm,
    verbose = True,
    process = Process.hierarchical
)

## Providing a Query

In [ ]:
# user_query = "WHen did the road trip start?"

Let the work Beginnnn!!!

In [ ]:
result = crew.kickoff()
print(result)

 [2024-07-28 17:30:23][DEBUG]: == Working Agent: Crew Manager
 [2024-07-28 17:30:23][INFO]: == Starting Task: 
        Provide a response to user's query based on content of the provided PDF.
        The agent1 will search through the PDF to find the relevant answers.
        The final answer MUST be clear & accurate, based on the content of the PDF.

        The User's query is:
        {user_query}
        


> Entering new CrewAgentExecutor chain...
The first step is to delegate the task to agent1 to search through the PDF and find the relevant answers to the user's query. I want to ensure that the agent is clear on the task and is able to provide an accurate response.

Action: Delegate work to coworker
Action Input: {"task": "Search through the PDF to find answers to the user's query", "context": "The user has asked a specific question, and the answer lies within the content of the provided PDF. It's important to read through the PDF thoroughly, understand the context, and extract 

Display the final result as Markdown

In [ ]:
print(result)

Unable to provide a response as the specific user's query and the PDF document were not provided. The issue has been communicated to the relevant parties for further action.


In [ ]:
# from IPython.display import Markdown
# Markdown(result)